In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import kerastuner as kt
from sklearn.model_selection import train_test_split

In [ ]:
#input with all maxima
df_spectra_all=pd.read_csv("spectrum_energy_input_numberOfPeaks.csv",index_col=[0])
x_data_string=df_spectra_all["all_maxima"].values

max_nr_of_max=df_spectra_all['no_of_max'].max()
len_array=len(df_spectra_all["all_maxima"].values)

print(max_nr_of_max,len_array)
x_data_padded=np.zeros((len_array,max_nr_of_max))

for i,x_string in enumerate(x_data_string):
    x_string=x_string.replace('\n','').replace('[','').replace(']','')
    x_split= x_string.split(' ')
    j=0
    for x in (x_split):
        if(x!=''):
            #print(x)
            x_data_padded[i,j]=float(x)
            j=j+1
    #print("cut")

y_data=df_spectra_all[["k6a1","k6a2","k11","k12","k9a1","k9a2"]].to_numpy()


y_data=y_data-0.5
y_data=y_data*4


data_train, data_test, labels_train, labels_test = train_test_split(x_data_padded, y_data, test_size=0.20, random_state=42)
   
print(data_train.shape)

def transform_data(df,index):
    data=df[index].to_numpy()
    new_data=np.zeros((len(data),max(data)+1))    
    for i in range(len(data)):
        new_data[i,data[i]]=1
    return new_data
    
df_spectra=pd.read_csv("spectrum_energy_input_numberOfPeaks.csv",index_col=[0])
df_new=df_spectra.loc[df_spectra["k11"] == 0]


x_data=transform_data(df_new,"no_of_max")
print(x_data[0:2])



y_data=df_spectra[["k6a1","k6a2","k11","k12","k9a1","k9a2"]].to_numpy()


y_data=y_data-0.5
y_data=y_data*4
print(y_data[0:10])


data_train, data_test, labels_train, labels_test = train_test_split(x_data, y_data, test_size=0.20, random_state=42)

print(data_train.shape)
print(labels_train.shape)

#df_training = df_spectra.sample(frac=0.8,random_state=10)
#df_testing=df_spectra.drop(df_training.index)
#df_spectra.head()

In [ ]:
feature_columns_A = []

no_of_max = tf.feature_column.numeric_column("all_maxima")
my_feature_layer_A = tf.keras.layers.DenseFeatures(no_of_max)

#transform data


In [ ]:

def plot_the_loss_curve(epochs, mse,val_mse):
    """Plot a curve of loss vs. epoch."""

    plt.figure()
    plt.xlabel("Epoch")
    plt.ylabel("Mean Squared Error")

    plt.plot(epochs, mse, label="Loss")
    plt.plot(epochs, val_mse, label="Val Loss")

    plt.legend()
    #plt.ylim([0, 15])
    plt.show()  

def create_model(my_learning_rate, my_feature_layer):
    model = tf.keras.models.Sequential()
    # Add the layer containing the feature columns to the model.
    # Define the first hidden layer with 10 nodes.   
   # model.add(my_feature_layer_A)
    #layers=[5,10]
    layers=[600,600,600,600]
    for layer in layers:
        model.add(tf.keras.layers.Dense(units = layer, activation = 'sigmoid',kernel_regularizer=tf.keras.regularizers.l1(0.0)))
    # Define the output layer.
    model.add(tf.keras.layers.Dense(units=6,  
                                    name='Output',activation="sigmoid"))#,kernel_regularizer=tf.keras.regularizers.l2(0.04)))                              
  
    model.compile(optimizer=tf.keras.optimizers.SGD(lr=my_learning_rate,momentum=0.1),
                loss="mean_squared_error",
                metrics=[tf.keras.metrics.MeanSquaredError()])

    return model



def train_model(model, x,y, epochs, label_name,
                batch_size=None):

    #features = {name:np.array(value) for name, value in dataset.items()}
    #label=dataset[label_name].to_numpy()
    history = model.fit(x=x, y=y, batch_size=batch_size,
                      epochs=epochs, shuffle=True, verbose=1,validation_split=0.2) 
    
    epochs = history.epoch
  
    df_hist = pd.DataFrame(history.history)
    #hist.head()
    #mse = hist["mean_squared_error"]
    mae = df_hist["mean_squared_error"].to_numpy()
    val_mae = df_hist["val_mean_squared_error"].to_numpy()
    return epochs,mae,val_mae

In [ ]:
# The following variables are the hyperparameters.
#data_train, data_test, labels_train, labels_test
learning_rate = 1e-1
epochs = 500
batch_size = 400

label_name = ["k6a1","k6a2","k11","k12","k9a1","k9a2"]

my_model = create_model(learning_rate, my_feature_layer_A)

epochs, mse,val_mse = train_model(my_model, data_train,labels_train, epochs, 
                          label_name, batch_size)


In [ ]:
#features = {name:np.array(value) for name, value in df_test.items()}
#label=df_test[label_name].to_numpy()
#data_train, data_test, labels_train, labels_test
plot_the_loss_curve(epochs, mse,val_mse)
evaluation=my_model.evaluate(x = data_test, y = labels_test, batch_size=batch_size)
predicted = my_model.predict(data_test)
print(evaluation)



df_test=pd.DataFrame(labels_test,columns=["k6a1_test","k6a2_test","k11_test","k12_test","k9a1_test","k9a2_test"])
df_predict=pd.DataFrame(predicted,columns=["k6a1_hat","k6a2_hat","k11_hat","k12_hat","k9a1_hat","k9a2_hat"])
pd.concat([df_test,df_predict], axis=1).head()

In [ ]:
print(data_test[0:5])
print(labels_test[0:5])


In [1]:
#new try with 2d Data:

import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import kerastuner as kt
from sklearn.model_selection import train_test_split


def get_array_from_strings(data_string, padded_array):
    use_paddad_array=padded_array.copy()
    for i,x_string in enumerate(data_string):
        x_string=x_string.replace('\n','').replace('[','').replace(']','')
        x_split= x_string.split(' ')
        j=0
        for x in (x_split):
            if(x!=''):
                #print(x)
                use_paddad_array[i,j]=float(x)
                j=j+1

    return use_paddad_array

H:\Users\Gwydion\anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
H:\Users\Gwydion\anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
H:\Users\Gwydion\anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
H:\Users\Gwydion\anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub

In [2]:
#input with all maxima
df_spectra_all=pd.read_csv("spectrum_energy_input_numberOfPeaks_with_intensity.csv",index_col=[0])
x_Energy_string=df_spectra_all["all_maxima"].values
x_Intensity_string=df_spectra_all["Intensity"].values



max_nr_of_max=df_spectra_all['no_of_max'].max()
len_array=len(df_spectra_all["all_maxima"].values)

print(max_nr_of_max,len_array)
x_data_padded=np.zeros((len_array,max_nr_of_max))


x_energy_padded=get_array_from_strings(x_Energy_string,x_data_padded)
x_Intensity_padded=get_array_from_strings(x_Intensity_string,x_data_padded)

#print(x_data_padded[:5])    
#print(x_energy_padded[:5])    
#print(x_Intensity_padded[:5])    

#make 2d Array from Energy and Intensity
x_data_2d =np.zeros((len(x_energy_padded),len(x_energy_padded[0]),2))
for i in range(len(x_energy_padded)):
    for j in range(len(x_energy_padded[i])):
        x_data_2d[i,j,0]=x_energy_padded[i,j]
        x_data_2d[i,j,1]=x_Intensity_padded[i,j]

print(x_data_2d[:5]) 

#convert y_data into 2d array where -1=[1,0] 0=[0,0] 1=[0,1]
y_data=df_spectra_all[["k6a1","k6a2","k11","k12","k9a1","k9a2"]].to_numpy()
y_data=y_data-0.5
y_data=y_data*4
y_data_2d =np.zeros((len(y_data),len(y_data[0]),2))
#print(y_data_2d.shape)
for i in range(len(y_data)):
    for j in range(len(y_data[i])):
        if(y_data[i,j]==-1):
            y_data_2d[i,j,0]=1
        elif(y_data[i,j]==1):
            y_data_2d[i,j,1]=1
            
#print(y_data[:5])
#print(y_data_2d[:5])
#this is with 2d label
#data_train, data_test, labels_train, labels_test = train_test_split(x_data_2d, y_data_2d, test_size=0.20, random_state=42)

#this is with 1d label
data_train, data_test, labels_train, labels_test = train_test_split(x_data_2d, y_data, test_size=0.20, random_state=42)




#maybe reshape data into [nr,x,y,channel=1?]   
print(data_train.shape)
print(labels_train.shape)

22 729
[[[-7.58000000e-01  5.20690186e+00]
  [-6.90000000e-01  6.88780025e+00]
  [-6.12000000e-01  3.55071480e+00]
  [-4.72000000e-01  6.73106847e+00]
  [-4.06000000e-01  6.52867583e+00]
  [-3.28000000e-01  2.03445274e+00]
  [-1.52000000e-01  5.86586448e+00]
  [-6.80000000e-02  4.57587849e+00]
  [-1.40000000e-02  3.76715204e+00]
  [ 1.34000000e-01  6.36630283e+00]
  [ 2.22000000e-01  4.15754817e+00]
  [ 2.94000000e-01  3.10619177e+00]
  [ 4.72000000e-01  5.88775394e+00]
  [ 5.24000000e-01  4.05543485e+00]
  [ 6.02000000e-01  5.13470720e+00]
  [ 7.48000000e-01  7.85833443e+00]
  [ 8.24000000e-01  2.62504489e+00]
  [ 8.82000000e-01  4.22528457e+00]
  [ 0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00]]

 [[-5.74000000e-01  2.38435403e+01]
  [-4.80000000e-02  1.10445002e+01]
  [ 3.40000000e-02  1.50444894e+01]
  [ 5.96000000e-01  1.12537878e+01]
  [ 6.58000000e-01  1.48091845e+01]
  [ 0.00000000e+00 

In [16]:

def plot_the_loss_curve(epochs, mse,val_mse):
    """Plot a curve of loss vs. epoch."""

    plt.figure()
    plt.xlabel("Epoch")
    plt.ylabel("Mean Squared Error")

    plt.plot(epochs, mse, label="Loss")
    plt.plot(epochs, val_mse, label="Val Loss")

    plt.legend()
    #plt.ylim([0, 15])
    plt.show()  

def create_2d_model(my_learning_rate, input_shape):
    model = tf.keras.models.Sequential()
    # Add the layer containing the feature columns to the model.
    # Define the first hidden layer with 10 nodes.   
    
    model.add(tf.keras.layers.Conv1D(32, 2, activation='relu',input_shape=input_shape))
   # model.add(tf.keras.layers.Conv1D(128, 4, activation='relu'))
   # model.add(tf.keras.layers.MaxPooling1D(2,2))

    model.add(tf.keras.layers.Conv1D(16, 2, activation='relu'))
   # model.add(tf.keras.layers.MaxPooling1D(2, 6))

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units = 6, activation = 'relu',kernel_regularizer=tf.keras.regularizers.l1(0.0)))

    
    
    
    model.compile(optimizer=tf.keras.optimizers.SGD(lr=my_learning_rate,momentum=0.1),
                loss="mean_squared_error",
                metrics=[tf.keras.metrics.MeanSquaredError()])

    return model



def train_model(model, x,y, epochs,
                batch_size=None):

    #features = {name:np.array(value) for name, value in dataset.items()}
    #label=dataset[label_name].to_numpy()
    history = model.fit(x=x, y=y, batch_size=batch_size,
                      epochs=epochs, shuffle=True, verbose=1,validation_split=0.1) 
    
    epochs = history.epoch
  
    df_hist = pd.DataFrame(history.history)
    #hist.head()
    #mse = hist["mean_squared_error"]
    mae = df_hist["mean_squared_error"].to_numpy()
    val_mae = df_hist["val_mean_squared_error"].to_numpy()
    return epochs,mae,val_mae

In [17]:
#data_train,data_test,labels_train,labels_test
input_shape=(data_train.shape[1:3])
print(input_shape)



(22, 2)


In [21]:
two_d_model=create_2d_model(0.01,(22,2))

In [ ]:
epochs, mse,val_mse =train_model(two_d_model,data_train,labels_train,200)

Epoch 1/200
17/17 [==============================] - ETA: 0s - loss: 1.4360 - mean_squared_error: 1.43 - 0s 5ms/step - loss: 0.7370 - mean_squared_error: 0.7370 - val_loss: 0.7013 - val_mean_squared_error: 0.7013
Epoch 2/200
17/17 [==============================] - ETA: 0s - loss: 0.6207 - mean_squared_error: 0.62 - 0s 2ms/step - loss: 0.6723 - mean_squared_error: 0.6723 - val_loss: 0.6984 - val_mean_squared_error: 0.6984
Epoch 3/200
17/17 [==============================] - ETA: 0s - loss: 0.6556 - mean_squared_error: 0.65 - 0s 2ms/step - loss: 0.6702 - mean_squared_error: 0.6702 - val_loss: 0.6969 - val_mean_squared_error: 0.6969
Epoch 4/200
17/17 [==============================] - ETA: 0s - loss: 0.6846 - mean_squared_error: 0.68 - 0s 2ms/step - loss: 0.6693 - mean_squared_error: 0.6693 - val_loss: 0.6956 - val_mean_squared_error: 0.6956
Epoch 5/200
17/17 [==============================] - ETA: 0s - loss: 0.6771 - mean_squared_error: 0.67 - 0s 2ms/step - loss: 0.6686 - mean_squared_e

17/17 [==============================] - ETA: 0s - loss: 0.6565 - mean_squared_error: 0.65 - 0s 2ms/step - loss: 0.6673 - mean_squared_error: 0.6673 - val_loss: 0.7019 - val_mean_squared_error: 0.7019
Epoch 40/200
17/17 [==============================] - ETA: 0s - loss: 0.6198 - mean_squared_error: 0.61 - 0s 2ms/step - loss: 0.6673 - mean_squared_error: 0.6673 - val_loss: 0.7022 - val_mean_squared_error: 0.7022
Epoch 41/200
17/17 [==============================] - ETA: 0s - loss: 0.6250 - mean_squared_error: 0.62 - 0s 2ms/step - loss: 0.6673 - mean_squared_error: 0.6673 - val_loss: 0.7023 - val_mean_squared_error: 0.7023
Epoch 42/200
17/17 [==============================] - ETA: 0s - loss: 0.6562 - mean_squared_error: 0.65 - 0s 2ms/step - loss: 0.6673 - mean_squared_error: 0.6673 - val_loss: 0.7023 - val_mean_squared_error: 0.7023
Epoch 43/200
17/17 [==============================] - ETA: 0s - loss: 0.6406 - mean_squared_error: 0.64 - 0s 2ms/step - loss: 0.6673 - mean_squared_error: 0.

17/17 [==============================] - ETA: 0s - loss: 0.6667 - mean_squared_error: 0.66 - 0s 2ms/step - loss: 0.6673 - mean_squared_error: 0.6673 - val_loss: 0.7037 - val_mean_squared_error: 0.7037
Epoch 78/200
17/17 [==============================] - ETA: 0s - loss: 0.6355 - mean_squared_error: 0.63 - 0s 2ms/step - loss: 0.6673 - mean_squared_error: 0.6673 - val_loss: 0.7038 - val_mean_squared_error: 0.7038
Epoch 79/200
17/17 [==============================] - ETA: 0s - loss: 0.7032 - mean_squared_error: 0.70 - 0s 2ms/step - loss: 0.6673 - mean_squared_error: 0.6673 - val_loss: 0.7039 - val_mean_squared_error: 0.7039
Epoch 80/200
17/17 [==============================] - ETA: 0s - loss: 0.6510 - mean_squared_error: 0.65 - 0s 2ms/step - loss: 0.6673 - mean_squared_error: 0.6673 - val_loss: 0.7041 - val_mean_squared_error: 0.7041
Epoch 81/200
17/17 [==============================] - ETA: 0s - loss: 0.6615 - mean_squared_error: 0.66 - 0s 2ms/step - loss: 0.6673 - mean_squared_error: 0.

17/17 [==============================] - ETA: 0s - loss: 0.6927 - mean_squared_error: 0.69 - 0s 2ms/step - loss: 0.6673 - mean_squared_error: 0.6673 - val_loss: 0.7039 - val_mean_squared_error: 0.7039
Epoch 116/200
17/17 [==============================] - ETA: 0s - loss: 0.6771 - mean_squared_error: 0.67 - 0s 2ms/step - loss: 0.6673 - mean_squared_error: 0.6673 - val_loss: 0.7039 - val_mean_squared_error: 0.7039
Epoch 117/200
17/17 [==============================] - ETA: 0s - loss: 0.6771 - mean_squared_error: 0.67 - 0s 2ms/step - loss: 0.6673 - mean_squared_error: 0.6673 - val_loss: 0.7040 - val_mean_squared_error: 0.7040
Epoch 118/200
17/17 [==============================] - ETA: 0s - loss: 0.6875 - mean_squared_error: 0.68 - 0s 2ms/step - loss: 0.6673 - mean_squared_error: 0.6673 - val_loss: 0.7041 - val_mean_squared_error: 0.7041
Epoch 119/200
17/17 [==============================] - ETA: 0s - loss: 0.6927 - mean_squared_error: 0.69 - 0s 2ms/step - loss: 0.6673 - mean_squared_error

17/17 [==============================] - ETA: 0s - loss: 0.6979 - mean_squared_error: 0.69 - 0s 2ms/step - loss: 0.6673 - mean_squared_error: 0.6673 - val_loss: 0.7045 - val_mean_squared_error: 0.7045
Epoch 154/200
17/17 [==============================] - ETA: 0s - loss: 0.6823 - mean_squared_error: 0.68 - 0s 2ms/step - loss: 0.6673 - mean_squared_error: 0.6673 - val_loss: 0.7045 - val_mean_squared_error: 0.7045
Epoch 155/200
17/17 [==============================] - ETA: 0s - loss: 0.6406 - mean_squared_error: 0.64 - 0s 2ms/step - loss: 0.6673 - mean_squared_error: 0.6673 - val_loss: 0.7046 - val_mean_squared_error: 0.7046
Epoch 156/200
17/17 [==============================] - ETA: 0s - loss: 0.6302 - mean_squared_error: 0.63 - 0s 2ms/step - loss: 0.6673 - mean_squared_error: 0.6673 - val_loss: 0.7048 - val_mean_squared_error: 0.7048
Epoch 157/200
17/17 [==============================] - ETA: 0s - loss: 0.6927 - mean_squared_error: 0.69 - 0s 2ms/step - loss: 0.6673 - mean_squared_error

In [ ]:
#features = {name:np.array(value) for name, value in df_test.items()}
#label=df_test[label_name].to_numpy()
#data_train, data_test, labels_train, labels_test
plot_the_loss_curve(epochs, mse,val_mse)
evaluation=two_d_model.evaluate(x = data_test, y = labels_test)
predicted = two_d_model.predict(data_test)
print(evaluation)



df_test=pd.DataFrame(labels_test,columns=["k6a1_test","k6a2_test","k11_test","k12_test","k9a1_test","k9a2_test"])
df_predict=pd.DataFrame(predicted,columns=["k6a1_hat","k6a2_hat","k11_hat","k12_hat","k9a1_hat","k9a2_hat"])
pd.concat([df_test,df_predict], axis=1).head()